In [30]:
import agentpy as ap
from owlready2 import *
from ultralytics import YOLO
import json

In [31]:
# Crear un nuevo archivo de ontología en memoria
onto = get_ontology("http://example.org/onto.owl")

with onto:
    class KnownObject(Thing):
        pass

    class SuspiciousObject(KnownObject):
        pass

    class SafeObject(KnownObject):
        pass

# Añadir ejemplos a la ontología
car = SafeObject("car")
toilet = SafeObject("toilet")
traffic = SuspiciousObject("traffic light")

onto.save(file="ontology.owl")


In [32]:
class CameraAgent(ap.Agent):
    
    def setup(self):
        self.safe_objects = set()  # Conjunto para objetos seguros
        self.suspicious_objects = set()  # Conjunto para objetos sospechosos
        self.load_ontology()
        self.model = YOLO("yolov8s.pt")  # Cargar el modelo YOLO solo una vez
    
    def load_ontology(self):
        # Cargar la ontología creada
        self.ontology = get_ontology("ontology.owl").load()
        for obj in self.ontology.SafeObject.instances():
            self.safe_objects.add(obj.name)
        for obj in self.ontology.SuspiciousObject.instances():
            self.suspicious_objects.add(obj.name)
    
    def detect_objects(self, image_path):
        """ Método para procesar objetos detectados """
        # Cargar y procesar la imagen usando el modelo YOLO
        results = self.model(image_path)
        
        # Extraer objetos detectados
        detected_objects = []
        json_result = results[0].tojson()
        
        json_result = json.loads(json_result)
        print(type(json_result[0]))
        print(json_result)
        for obj in json_result:
            detected_objects.append(obj["name"])
        
        # Analizar los objetos detectados
        suspicious_detected = []
        for obj in detected_objects:
            if obj in self.suspicious_objects:
                suspicious_detected.append(obj)
                print(f"Alerta: Objeto sospechoso detectado - {obj}")
            elif obj in self.safe_objects:
                print(f"Objeto seguro detectado - {obj}")
            else:
                suspicious_detected.append(obj)
                print(f"Alerta: Objeto desconocido detectado - {obj}")
        
        return suspicious_detected

In [33]:
class MyModel(ap.Model):

    def setup(self):
        # Called at the start of the simulation
        #self.agents = ap.AgentList(self, self.p.agents, CameraAgent)
        pass

In [34]:
# Ejemplo de uso
camera_agent = CameraAgent(MyModel())
suspicious_objects = camera_agent.detect_objects("aerial-drone-pov.jpg")
print(f"Objetos sospechosos detectados: {suspicious_objects}")


image 1/1 c:\Users\phere\Documents\TEC\5toSemestre\Reto_multiagentes\aerial-drone-pov.jpg: 416x640 2 cars, 1 traffic light, 1 toilet, 185.5ms
Speed: 2.0ms preprocess, 185.5ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
<class 'dict'>
[{'name': 'car', 'class': 2, 'confidence': 0.57702, 'box': {'x1': 223.9494, 'y1': 33.70423, 'x2': 253.50323, 'y2': 63.26784}}, {'name': 'toilet', 'class': 61, 'confidence': 0.33642, 'box': {'x1': 30.94511, 'y1': 0.31304, 'x2': 608.98364, 'y2': 383.96381}}, {'name': 'car', 'class': 2, 'confidence': 0.31546, 'box': {'x1': 288.68121, 'y1': 0.09993, 'x2': 305.10846, 'y2': 14.2923}}, {'name': 'traffic light', 'class': 9, 'confidence': 0.29761, 'box': {'x1': 135.03931, 'y1': 106.3971, 'x2': 184.41, 'y2': 187.78621}}]
Objeto seguro detectado - car
Objeto seguro detectado - toilet
Objeto seguro detectado - car
Alerta: Objeto sospechoso detectado - traffic light
Objetos sospechosos detectados: ['traffic light']
